In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime
from scipy import stats
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dfs_paths = ["../input/brazilian-ecommerce/olist_customers_dataset.csv", "../input/brazilian-ecommerce/olist_geolocation_dataset.csv",
            "../input/brazilian-ecommerce/olist_order_items_dataset.csv", "../input/brazilian-ecommerce/olist_order_payments_dataset.csv",
            "../input/brazilian-ecommerce/olist_order_reviews_dataset.csv", "../input/brazilian-ecommerce/olist_orders_dataset.csv",
            "../input/brazilian-ecommerce/olist_products_dataset.csv", "../input/brazilian-ecommerce/olist_sellers_dataset.csv",
            "../input/brazilian-ecommerce/product_category_name_translation.csv"]

In [ ]:
# Set dfs
customers_df = pd.read_csv("../input/brazilian-ecommerce/olist_customers_dataset.csv")
geo_df = pd.read_csv("../input/brazilian-ecommerce/olist_geolocation_dataset.csv")
orderitem_df = pd.read_csv("../input/brazilian-ecommerce/olist_order_items_dataset.csv")
orderpay_df = pd.read_csv("../input/brazilian-ecommerce/olist_order_payments_dataset.csv")
orderreviews_df = pd.read_csv("../input/brazilian-ecommerce/olist_order_reviews_dataset.csv")
orders_df = pd.read_csv("../input/brazilian-ecommerce/olist_orders_dataset.csv")
products_df = pd.read_csv("../input/brazilian-ecommerce/olist_products_dataset.csv")
sellers_df = pd.read_csv("../input/brazilian-ecommerce/olist_sellers_dataset.csv")
categname_df = pd.read_csv("../input/brazilian-ecommerce/product_category_name_translation.csv")
pd.set_option('display.max_columns', 500)

In [ ]:
# Change cols names before merging 
customers_df.rename(columns={"customer_zip_code_prefix": "zip_code"}, inplace=True)
geo_df.rename(columns={"geolocation_zip_code_prefix": "zip_code"}, inplace=True)

In [ ]:
# Join datasets
data = orders_df.merge(customers_df, on="customer_id").merge(orderitem_df, on="order_id").merge(products_df, on="product_id").merge(categname_df, on="product_category_name").merge(orderpay_df, on="order_id").merge(sellers_df, on="seller_id").merge(orderreviews_df, on="order_id")

In [ ]:
data.head()

We have duplicated order_ids. This is due to the fact that the same order can be paid by multiple payment methods. 

In [ ]:
# Null values 
(data.isna().sum() / len(data) ).sort_values(ascending=False)

In [ ]:
# Main stats
data.describe()

# **How are review scores distributed?**

In [ ]:
sns.countplot(data["review_score"])
data["review_score"].value_counts() / data["review_score"].count() * 100

Over 75% of customers gave a score equal or greater than 4. 12.5 % gave a score of 1 and about 12% gave a score of 3 or 2. 

# **Proportion of customers generating most of the revenue:**

In [ ]:
# Customers wiht highest cum orders (in payments)
top_customers = data.groupby("customer_unique_id")["payment_value"].sum().reset_index().sort_values("payment_value", ascending=False)
top_customers.rename(columns={"payment_value":"total_paid"}, inplace=True)

In [ ]:
top_customers["% of Total Sales"] = (top_customers["total_paid"] / top_customers["total_paid"].sum()) * 100
top_customers["Cum % of Total Sales"] = top_customers["% of Total Sales"].cumsum() 
#sns.displot(top_customers["Cum % of Total Sales"])
ax = sns.lineplot(x=range(1,len(top_customers)+1), y="Cum % of Total Sales", data=top_customers)
ax.set_xlabel("N° of Customers")
ax.set_title("% Contribution to Sales by number of customers")

40000 customers (about 42% of total customers) contribute to approximately 80% of total sales.

# **Who are the top customers?**

In [ ]:
top_customers.rename(columns={"payment_value" : "total_paid"}, inplace=True)
ax = sns.barplot(x="total_paid", y="customer_unique_id", data=top_customers[:10])
ax.set_title("Top 10 Customers by total paid")

# **Top cites by number of orders by state**

In [ ]:
top_orders_cities = data.groupby("customer_state")["order_id"].count().reset_index().sort_values("order_id", ascending=False)
top_orders_cities.rename(columns={"order_id":"count"}, inplace=True)
ax = sns.barplot(x="count", y="customer_state", data=top_orders_cities[:10])
ax.set_title("TOP 10 states by num of orders")

# **Cities with highest revenue generation**

In [ ]:
top_ordersbyvalue_cities = data.groupby("customer_city")["payment_value"].sum().reset_index().sort_values("payment_value", ascending=False)
top_ordersbyvalue_cities["% of Total Payments"] = (top_ordersbyvalue_cities["payment_value"] / top_ordersbyvalue_cities["payment_value"].sum()) * 100
top_ordersbyvalue_cities["Cum % of Total Payments"] = top_ordersbyvalue_cities["% of Total Payments"].cumsum() 

In [ ]:
ax = sns.barplot(x="% of Total Payments", y="customer_city", data=top_ordersbyvalue_cities[:10])
ax.set_title("TOP 10 cities by revenue generation")

In [ ]:
ax = sns.lineplot(x=range(1,len(top_ordersbyvalue_cities)+1), y="Cum % of Total Payments", data=top_ordersbyvalue_cities)
ax.set_xlabel("N° of cities")
ax.set_title("% Contribution to Sales by number of cities")

In [ ]:
print("Number of cities contributing to 80% of total sales:",
      len(top_ordersbyvalue_cities[top_ordersbyvalue_cities["Cum % of Total Payments"] <= 80]),
      "or in %:",
      (len(top_ordersbyvalue_cities[top_ordersbyvalue_cities["Cum % of Total Payments"] <= 80]) / len(top_ordersbyvalue_cities)) * 100)

# **How do orders vary across time?**

In [ ]:
# Total orders by hour and dow
# But before I need to convert the dates cols to datetime 
datesCols = ["order_purchase_timestamp", "order_approved_at", "order_delivered_carrier_date", 
            "order_delivered_customer_date", "order_estimated_delivery_date", "shipping_limit_date", 
            "review_creation_date", "review_answer_timestamp"]

for col in datesCols:
    data[col] = pd.to_datetime(data[col])

In [ ]:
# Orders by hour
orders_df["order_purchase_timestamp"] = pd.to_datetime(orders_df["order_purchase_timestamp"])
orderbyhour = orders_df.groupby(orders_df["order_purchase_timestamp"].dt.hour)["order_id"].count().reset_index().sort_values(by="order_purchase_timestamp", ascending=False)
orderbyhour.rename(columns={"order_id":"Total Orders", "order_purchase_timestamp": "Hour of Day"}, inplace=True)

In [ ]:
# Visualize data
ax = sns.barplot(x="Hour of Day", y="Total Orders", data=orderbyhour)
ax.set_title("N° of Orders by hour")

In [ ]:
# Orders by day of the week
orderbydow = data.groupby(data["order_purchase_timestamp"].dt.day_name())["order_id"].count().reset_index()
orderbydow.rename(columns={"order_id":"Total Orders", "order_purchase_timestamp": "Weekday Name"}, inplace=True)
orderbydow = orderbydow.sort_values(by="Total Orders", ascending=False)

In [ ]:
plt.figure(figsize = (9,8))
ax = sns.barplot(x="Weekday Name", y="Total Orders", data=orderbydow)
ax.set_xlabel('')
ax.set_title("N° of Orders by DOW")

# **How products are rated ?**

In [ ]:
# Best and worst rated products
plt.figure(figsize=(12,12))
reviewsocres = data.groupby("product_category_name_english")["review_score"].agg(["mean", "count"]).sort_values(by="mean",ascending=False)
#ax = sns.barplot(y=bestrated.index, x=bestrated.values)
# Get only products with 30 or more reviews in order to have a more realistic idea about the rating
bestrated = reviewsocres[reviewsocres["count"]>=30][:10]
bestrated

In [ ]:
# Bottom 10 Products by review socre
worstrated = reviewsocres[reviewsocres["count"]>=30].sort_values(by='mean')[:10]
worstrated

# **Does payment method affect order status?**

In [ ]:
# For example: does paying cash increase order cancelation
cashvscancel = pd.crosstab(data["payment_type"], data["order_status"])
cashvscancel = cashvscancel[["canceled", "delivered"]]
cashvscancel["% Canceled"] = (cashvscancel["canceled"] / cashvscancel["delivered"] ) * 100
cashvscancel["Avg Cancelation Rate"] = (len(data[data["order_status"] == "canceled"]) / len(data[data["order_status"] == "delivered"])) * 100
cashvscancel


# **Is there any relationship between delivery time and review scores**

In [ ]:
# Add delta column which computes the time it took for the order to get delivered
data["TimeToDeliveryinHours"] = (data["order_delivered_customer_date"] - data["order_purchase_timestamp"])
data["TimeToDeliveryinHours"] = data["TimeToDeliveryinHours"].apply(lambda x: x.total_seconds())
data["TimeToDeliveryinHours"] = round((data["TimeToDeliveryinHours"] / 3600) / 24, 2)
data.rename(columns={"TimeToDeliveryinHours" : "TimeToDeliveryinDays"}, inplace=True)

In [ ]:
# Main stats of deliverytime
data[["TimeToDeliveryinDays"]].describe()

In [ ]:
sns.boxplot(x="review_score", y="TimeToDeliveryinDays", data=data)

In [ ]:
# Outliers removal
q_high = data["TimeToDeliveryinDays"].quantile(0.95)
data_no_outliers = data[data["TimeToDeliveryinDays"] < q_high]
sns.boxplot(x="review_score", y="TimeToDeliveryinDays", data=data_no_outliers)

# **What are the sellers' cities with lowest/highest delivery time?**

In [ ]:
# Let's see sellers with best deliverytime
sellersdeliverytime = data.groupby("seller_city")["TimeToDeliveryinDays"].agg(["min", "max", "mean", "std", "count" ]).dropna().sort_values("mean").reset_index()
# Filter for sellers with 30 or more orders in their history
sellersdeliverytime = sellersdeliverytime[sellersdeliverytime["count"]>=30]

In [ ]:
fastestdeliverysellers = sellersdeliverytime[:10]
slowestdeliverysellers = sellersdeliverytime.sort_values("mean", ascending=False)[:10]

In [ ]:
# Fastest delivery sellers
fastestdeliverysellers

In [ ]:
# Join average review score to table above
avg_review_score_seller = data.groupby("seller_city")["review_score"].mean().dropna().sort_values(ascending=False).reset_index()

In [ ]:
sellerPerf = sellersdeliverytime.merge(avg_review_score_seller, on="seller_city")

In [ ]:
# Relationship between average timetodelivery and average review score
ax = sns.regplot(x="mean", y="review_score", data=sellerPerf)
ax.set_xlabel("")

We can clearly see that there is a moderate negative relationship between the time it takes for sellers to deliver their orders and the review they get.

In [ ]:
# Slowest delivery sellers
slowestdeliverysellers

# **States with highest/lowest time to delivery**

In [ ]:
highestTTDstates = data.groupby("customer_state")["TimeToDeliveryinDays"].mean().dropna().sort_values(ascending=False).reset_index()
highestTTDstates = highestTTDstates[:10]
ax = sns.barplot(y="customer_state", x="TimeToDeliveryinDays", data=highestTTDstates)

In [ ]:
lowestTTDstates = data.groupby("customer_state")["TimeToDeliveryinDays"].mean().dropna().sort_values(ascending=True).reset_index()
lowestTTDstates = lowestTTDstates[:10]
ax = sns.barplot(y="customer_state", x="TimeToDeliveryinDays", data=lowestTTDstates)

# **How does average deliverytime vary across time?**

In [ ]:
deliverytimevstime = data.groupby(data["order_purchase_timestamp"].dt.year)["TimeToDeliveryinDays"].median().dropna()
plt.figure(figsize=(9,6))
deliverytimevstime.plot(kind="bar")
plt.xlabel("")
plt.ylabel("Median deliveryTime in days")

In [ ]:
scorevstime = data.groupby(data["order_purchase_timestamp"].dt.year)["review_score"].mean().dropna()
plt.figure(figsize=(9,6))
scorevstime.plot(kind="bar")
plt.xlabel("")
plt.ylabel("Mean review_score")

Customers are giving better scores in 2017 and 2018 than in 2016. 

In [ ]:
top_categ_by_revenue = data.groupby("product_category_name_english").agg({'order_id':'nunique','payment_value':'sum'}).sort_values("payment_value", ascending=False)[:10]
top_categ_by_revenue.rename(columns={"order_id":"NumOfOrders", "payment_value":"Revenues"}, inplace=True)

In [ ]:
top_categ_by_revenue

In [ ]:
# Calculate recency 
df_recency = data.groupby(by='customer_unique_id', as_index=False)['order_purchase_timestamp'].max()
df_recency.rename(columns={"order_purchase_timestamp":"LastPurchaseDate"}, inplace=True)
df_recency["LastPurchaseDate"] = df_recency["LastPurchaseDate"].dt.date
# Get recent invoice date and use it to calculate recency
recent_date = data['order_purchase_timestamp'].dt.date.max()
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply(lambda x: (recent_date - x).days)

In [ ]:
df_recency.head()

In [ ]:
# Calculating frequency
frequency_df = data.groupby(["customer_unique_id"]).agg({"order_id":"nunique"}).reset_index()
frequency_df.rename(columns={"order_id":"Frequency"}, inplace=True)
frequency_df.head()

In [ ]:
# Calculating monetray value
monetary_df = data.groupby('customer_unique_id', as_index=False)['payment_value'].sum()
monetary_df.columns = ['customer_unique_id', 'Monetary']
monetary_df.head()

In [ ]:
# Merging dfs
rf_df = df_recency.merge(frequency_df, on='customer_unique_id')
rfm_df = rf_df.merge(monetary_df, on='customer_unique_id').drop(columns='LastPurchaseDate')
rfm_df.head()

In [ ]:
# Remove zeros from data before log trans
rfm_df[rfm_df.columns[1:]] = rfm_df[rfm_df.columns[1:]].applymap(lambda x: 1 if x ==0 else x)

In [ ]:
# Running K-Means algo to cluster our data
# Let's check if data is skewed first
def check_skew(df_skew, column):
    skew = stats.skew(df_skew[column])
    skewtest = stats.skewtest(df_skew[column])
    plt.title('Distribution of ' + column)
    sns.distplot(df_skew[column])
    plt.show()
    print("{}'s: Skew: {}, : {}".format(column, skew, skewtest))
    return

In [ ]:
for col in rfm_df.columns[1:]:
    check_skew(rfm_df, col)
    

In [ ]:
# Perform log transformation 
rfm_df_log = rfm_df.copy()
for c in rfm_df.columns[2:]:
    rfm_df_log[c] = np.log10(rfm_df_log[c])

In [ ]:
for col in rfm_df.columns[1:]:
    check_skew(rfm_df_log, col)

In [ ]:
scaler = StandardScaler()
scaler.fit(rfm_df_log.drop("customer_unique_id", axis=1))
RFM_Table_scaled = scaler.transform(rfm_df_log.drop("customer_unique_id", axis=1))
# Put data into a df
RFM_Table_scaled = pd.DataFrame(RFM_Table_scaled, columns=rfm_df_log.columns[1:])

In [ ]:
RFM_Table_scaled.head()

In [ ]:
# Training k-means model

distortions = []
K = range(1,10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(RFM_Table_scaled)
    distortions.append(kmeanModel.inertia_)

plt.figure(figsize=(9,8))
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
# Train the model on 4 clusters
kmean_model = KMeans(n_clusters=4, random_state=5)
kmean_y = kmean_model.fit_predict(RFM_Table_scaled)
# Add labels to df
rfm_df['Cluster'] = kmean_model.labels_

In [ ]:
# Function to visualize clusters
def rfm_values(df):
    df_new = df.groupby(['Cluster']).agg({
            'Recency': 'mean',
            'Frequency': 'mean',
            'Monetary': ['mean', 'count']
        }).round(0)

    return df_new

In [ ]:
rfm_values(rfm_df)